# Hearing a Rectangle

In [46]:
# Imports
#
# C:\Users\sulta\AppData\Local\Programs\Python\Python310\Scripts\pip install scipy
#
import numpy as np
from scipy import stats, optimize
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from scipy.optimize import shgo

In [49]:
# Parameters of the "true" rectangle
a_true = 1.0
b_true = 3.0
pi = np.pi

# sollte locker für 1000 Eigenwerte reichen
m_max, n_max = 800, 800

ms = np.arange(1, m_max + 1)
ns = np.arange(1, n_max + 1)
M, N = np.meshgrid(ms, ns, indexing="ij")

# Dirichlet-Eigenwerte: λ_{m,n} = π²(m²/a² + n²/b²)
lambdas = pi**2 * (M**2 / a_true**2 + N**2 / b_true**2)

# Flatten & sort
l_flat = lambdas.ravel()
sort_idx = np.argsort(l_flat)
l_sorted = l_flat[sort_idx]

Nvals = 10000
eigvals = l_sorted[:Nvals]

print("Erste 12 Eigenwerte (gerundet):")
print([round(float(x), 4) for x in eigvals[:12]])

Erste 12 Eigenwerte (gerundet):
[10.9662, 14.2561, 19.7392, 27.4156, 37.2852, 40.575, 43.8649, 49.348, 49.348, 57.0244, 63.6041, 66.894]


In [65]:
def reconstruct_aspect_ratio_final_targeted(eigvals, N_total_fit=9500, N_modes=300):
    """
    Rekonstruiert die Seitenlängen (a, b) und das Seitenverhältnis R = a/b 
    unter Verwendung eines extrem hochauflösenden, gezielten L2-Grid Search 
    im Intervall [0.30, 0.40], um das globale Minimum bei R=1/3 zu isolieren.
    
    Args:
        eigvals (list oder np.ndarray): Die sortierten Eigenwerte (λ_k).
        N_total_fit (int): Anzahl der Eigenwerte, die für den Fit verwendet werden.
        N_modes (int): Maximale m, n, die zur Generierung der λ_m,n verwendet werden.

    Returns:
        tuple: (rekonstruiertes_a, rekonstruiertes_b, rekonstruiertes_R)
    """
    pi = np.pi
    
    # Zum Vergleich (anhand Ihrer bekannten Parameter)
    a_true, b_true = 1.0, 3.0

    N_total = min(len(eigvals), N_total_fit)
    eigvals = np.array(eigvals[:N_total])
    k_vals = np.arange(1, N_total + 1)
    
    # Warnungen unterdrücken
    filterwarnings("ignore", category=np.VisibleDeprecationWarning)
    filterwarnings("ignore", category=RuntimeWarning)

    print("=========================================================================")
    print("Rekonstruktion der Seitenlängen (FINAL: ULTRA-FEINE SUCHE)")
    print("=========================================================================")
    
    # ----------------------------------------------------
    ## 1. Schritt: Rekonstruktion der Fläche (A = ab)
    # ----------------------------------------------------
    start_index_A = int(0.3 * N_total)
    slope_A, _, _, _, _ = stats.linregress(eigvals[start_index_A:], k_vals[start_index_A:])
    A_hat = 4 * pi * slope_A
    print(f"--- Schritt 1: Fläche (Area) ---")
    print(f"Rekonstruierte Fläche A = a*b ≈ {A_hat:.4f} (Wahr: {a_true*b_true:.1f})")

    # ----------------------------------------------------
    ## 2. Schritt: Ultra-Feine L2-Grid Search für R
    # ----------------------------------------------------
    print(f"\n--- Schritt 2: Ultra-Feine Grid-Search für R (Bereich [0.30, 0.40]) ---")
    
    ms_local = np.arange(1, N_modes + 1)
    ns_local = np.arange(1, N_modes + 1)
    M, N = np.meshgrid(ms_local, ns_local, indexing="ij")
    
    def calculate_L2_error_ultra_fine(R):
        """Kostenfunktion: L2-Abweichung über fast alle Eigenwerte."""
        if R <= 1e-6 or R > 1.0: return 1e20
        
        a = np.sqrt(A_hat * R)
        b = np.sqrt(A_hat / R)
        
        lambda_gen = pi**2 * (M**2 / a**2 + N**2 / b**2)
        l_gen_sorted = np.sort(lambda_gen.ravel())[:N_total]
        
        # Fehler über den Großteil des Spektrums (ab dem 10. Eigenwert)
        start_index = 10 
        diff = eigvals[start_index:] - l_gen_sorted[start_index:]
        return np.sum(diff**2)

    # WICHTIG: 2000 Punkte in einem sehr engen Intervall, um das schmale globale Minimum zu treffen.
    R_grid = np.linspace(0.30, 0.40, 2000) 
    
    errors = np.array([calculate_L2_error_ultra_fine(R) for R in R_grid])
    
    min_idx = np.argmin(errors)
    R_hat = R_grid[min_idx]
    
    # ----------------------------------------------------
    ## 3. Schritt: Endergebnis
    # ----------------------------------------------------
    
    a_hat = np.sqrt(A_hat * R_hat)
    b_hat = np.sqrt(A_hat / R_hat)

    print(f"Bester Fit für Seitenverhältnis R = a/b ≈ {R_hat:.4f} (Gefunden im gezielten Bereich)")
    
    print(f"\n--- Schritt 3: Endergebnis ---")
    print(f"Rekonstruierte Seitenlängen (geordnet): (a, b) ≈ ({a_hat:.4f}, {b_hat:.4f})")
    
    a_true_final, b_true_final = min(a_true, b_true), max(a_true, b_true)
    print(f"Wahre Seitenlängen (geordnet): (a, b) = ({a_true_final:.1f}, {b_true_final:.1f})")
    print(f"Abweichung: |a_final - a_true|: {abs(a_hat - a_true_final):.4f}")
    print(f"Abweichung: |b_final - b_true|: {abs(b_hat - b_true_final):.4f}")
    
    print("\n--- Rekonstruktion abgeschlossen ---")

    return a_hat, b_hat, R_hat

In [66]:
a_final, b_final, R_final = reconstruct_aspect_ratio_confirmed(eigvals)
print(f"Ergebnis: a={a_final:.4f}, b={b_final:.4f}, R={R_final:.4f}")

Rekonstruktion der Seitenlängen (FINAL: BESTÄTIGTE R-SUCHE)
--- Schritt 1: Fläche (Area) ---
Rekonstruierte Fläche A = a*b ≈ 2.9747 (Wahr: 3.0)

--- Schritt 2: Hochauflösende Grid-Search für R (Bereich [0.25, 0.45]) ---
Bester Fit für Seitenverhältnis R = a/b ≈ 0.4500 (Gefunden im gezielten Bereich)

--- Schritt 3: Endergebnis ---
Rekonstruierte Seitenlängen (geordnet): (a, b) ≈ (1.1570, 2.5711)
Wahre Seitenlängen (geordnet): (a, b) = (1.0, 3.0)
Abweichung: |a_final - a_true|: 0.1570
Abweichung: |b_final - b_true|: 0.4289

--- Rekonstruktion abgeschlossen ---
Ergebnis: a=1.1570, b=2.5711, R=0.4500
